<a href="https://colab.research.google.com/github/YuhaiW/00/blob/main/model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Putting it all together 


In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

# check pytorch  Version 
torch.__version__


'1.13.1+cu116'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device:{device}")

Using device:cuda


## 1. Data preparetion 

In [ ]:
# create some data

weight = 0.7
bias = 0.3

# create range values 
start = 0
end = 1
step = 0.02

# Create X and y

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]), tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [ ]:
## split data 
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

## building pytorch linear model

In [ ]:
# create a linear model by subclassing nn.modeule 
class LinearRegressionModelV2(nn.Module):
  def __init__(self):
    super().__init__()
    #use nn.Linear() for creating the mpodel parameters
    self.linear_layer = nn.Linear(in_features=1,
                                  out_features=1)
  def forward(self, x : torch.Tensor) -> torch.Tensor:
    return self.linear_layer(x)


# set seed
torch.manual_seed(42)
model_1 = LinearRegressionModelV2()
model_1, model_1.state_dict()

(LinearRegressionModelV2(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
              ('linear_layer.bias', tensor([0.8300]))]))

In [ ]:
## check the model device

next(model_1.parameters()).device

device(type='cpu')

In [ ]:
# set the model to use the target device
model_1.to(device)
next(model_1.parameters()).device

device(type='cuda', index=0)

## 2. Training
- loss function 
- optimizer
- training loop
- testing loop

In [ ]:
## Setup loss function 
loss_fn = nn.L1Loss()

## setup out optimizer
optimizer1 = torch.optim.SGD(params = model_1.parameters(), 
                             lr = 0.01)




In [ ]:
## taining loop

torch.manual_seed(42)

epochs = 200

## put data in the same device 
X_train  = X_train.to(device)
y_train  = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)



for epoch in range(epochs):
  model_1.train()
  y_pred = model_1(X_train)
  loss = loss_fn(y_pred, y_train)
  optimizer1.zero_grad()
  loss.backward()
  optimizer1.step()

  ## testing 
  model_1.eval()
  with torch.inference_mode():
    test_pred = model_1(X_test)
    test_loss = loss_fn(test_pred, y_test)

  #print out what's happening 

  if epoch % 20 ==0:
    print(f"epoch: {epoch} | loss: {loss} | test_loss {test_loss}")


epoch: 0 | loss: 0.5551779866218567 | test_loss 0.5739762187004089
epoch: 20 | loss: 0.3247582018375397 | test_loss 0.30455657839775085
epoch: 40 | loss: 0.09433845430612564 | test_loss 0.03513690456748009
epoch: 60 | loss: 0.019956795498728752 | test_loss 0.045803118497133255
epoch: 80 | loss: 0.013089174404740334 | test_loss 0.02994490973651409
epoch: 100 | loss: 0.006215683650225401 | test_loss 0.014086711220443249
epoch: 120 | loss: 0.0012645035749301314 | test_loss 0.013801801018416882
epoch: 140 | loss: 0.0012645035749301314 | test_loss 0.013801801018416882
epoch: 160 | loss: 0.0012645035749301314 | test_loss 0.013801801018416882
epoch: 180 | loss: 0.0012645035749301314 | test_loss 0.013801801018416882


In [ ]:
model_1.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.6968]], device='cuda:0')),
             ('linear_layer.bias', tensor([0.3025], device='cuda:0'))])

In [ ]:
## making and evaluating predictions 
# turn mdoel into evaluation mode 
model_1.eval()

# make predictions on the test data 
with torch.inference_mode():
  y_preds = model_1(X_test)
y_preds

tensor([[0.8600],
        [0.8739],
        [0.8878],
        [0.9018],
        [0.9157],
        [0.9296],
        [0.9436],
        [0.9575],
        [0.9714],
        [0.9854]], device='cuda:0')

In [ ]:
## saving and loading a trained model 


In [ ]:
from pathlib import Path

## 1. create dictionary
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)


## 2. Create model save path

MODEL_NAME = "01_model.pt"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME


## 3.  Save the model state dict 
print(f"saving model to :{MODEL_SAVE_PATH}")
torch.save(obj = model_1.state_dict(),
           f = MODEL_SAVE_PATH)

saving model to :models/01_model.pt
